In [200]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mne
import glob

%matplotlib notebook

In [201]:
data_dir = '/Users/nicolaspiron/Documents/PULSATION/Python_MNE/preproc/n2pc_out/data/epochs/'
file = data_dir+'S18_n2pc_total_epochs.fif'
epochs = mne.read_epochs(file)

Reading /Users/nicolaspiron/Documents/PULSATION/Python_MNE/preproc/n2pc_out/data/epochs/S18_n2pc_total_epochs.fif ...
    Found the data of interest:
        t =    -200.00 ...    1000.00 ms
        0 CTF compensation matrices available


/var/folders/4f/_jg1y72j6_ggv65n730flb280000gn/T/ipykernel_1297/3019245996.py:3: RuntimeWarning: This filename (/Users/nicolaspiron/Documents/PULSATION/Python_MNE/preproc/n2pc_out/data/epochs/S18_n2pc_total_epochs.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs = mne.read_epochs(file)


Not setting metadata
211 matching events found
No baseline correction applied
0 projection items activated


In [203]:
# create a function that classes the epoched file in different smaller files by condition

def sort_epochs(epochs):
    
    event_ids = epochs.event_id
    
    sorted_epochs = []
    
    for event in event_ids.keys():
        
        condition = epochs[event]
        sorted_epochs.append(condition)

    # concatenate the the files when the target is on the same side but the distractor is bot or top
    
    dis_vert_target_l = mne.concatenate_epochs([sorted_epochs[0], sorted_epochs[4]])
    dis_vert_target_r = mne.concatenate_epochs([sorted_epochs[1], sorted_epochs[5]])
    
    sorted_epochs.append(dis_vert_target_l)
    sorted_epochs.append(dis_vert_target_r)
    
    # remove useless files
    
    indices_to_remove = [0, 1, 4, 5]
    
    # reverse to avoid index shifting issues
    
    indices_to_remove.sort(reverse=True)
    for index in indices_to_remove:
        del sorted_epochs[index]
    
    return sorted_epochs  

In [204]:
sorted_epochs = sort_epochs(epochs)
sorted_epochs

Not setting metadata
36 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
36 matching events found
Applying baseline correction (mode: mean)


[<EpochsFIF |  34 events (all good), -0.2 - 1 sec, baseline -0.2 – 0 sec, ~11.2 MB, data loaded,
  'no_dis/target_l': 34>,
 <EpochsFIF |  34 events (all good), -0.2 - 1 sec, baseline -0.2 – 0 sec, ~11.2 MB, data loaded,
  'no_dis/target_r': 34>,
 <EpochsFIF |  35 events (all good), -0.2 - 1 sec, baseline -0.2 – 0 sec, ~11.5 MB, data loaded,
  'dis_right/target_l': 35>,
 <EpochsFIF |  36 events (all good), -0.2 - 1 sec, baseline -0.2 – 0 sec, ~11.8 MB, data loaded,
  'dis_left/target_r': 36>,
 <EpochsArray |  36 events (all good), -0.2 - 1 sec, baseline -0.2 – 0 sec, ~11.8 MB, data loaded,
  'dis_top/target_l': 17
  'dis_bot/target_l': 19>,
 <EpochsArray |  36 events (all good), -0.2 - 1 sec, baseline -0.2 – 0 sec, ~11.8 MB, data loaded,
  'dis_top/target_r': 20
  'dis_bot/target_r': 16>]

In [205]:
def compute_alpha_by_side(sorted_epochs):

    freqs = np.arange(8, 13)
    right_elecs=[ 'P8', 'P10', 'PO8']
    left_elecs=['P7', 'P9', 'PO7']

    n_cycles = freqs / 2.
    time_bandwidth = 4.
    baseline = None  # no baseline correction
    n_jobs = 1  # number of parallel jobs to run
    
    right_power_list = []
    left_power_list = []

    # compute alpha power and append it to right/left_power_list
    for i in range(len(conditions)):
        right_power = mne.time_frequency.tfr_morlet(sorted_epochs[i], freqs=freqs, n_cycles=n_cycles, picks=right_elecs,
                                                use_fft=True, return_itc=False, decim=1,
                                                n_jobs=n_jobs, verbose=True)
        right_power_mean = right_power.to_data_frame()[right_elecs].mean(axis=1).mean(axis=0)
        right_power_list.append(right_power_mean)

        left_power= mne.time_frequency.tfr_morlet(sorted_epochs[i], freqs=freqs, n_cycles=n_cycles, picks=left_elecs,
                                                use_fft=True, return_itc=False, decim=1,
                                                n_jobs=n_jobs, verbose=True)
        left_power_mean = left_power.to_data_frame()[left_elecs].mean(axis=1).mean(axis=0)
        left_power_list.append(left_power_mean)
    
    return right_power_list, left_power_list

In [206]:
right_power_list, left_power_list = compute_alpha_by_side(sorted_epochs)
right_power_list

[3.2612289125729613e-10,
 3.3128462461061294e-10,
 3.4074995964706485e-10,
 3.00410816402673e-10,
 3.456236359488997e-10,
 3.2317055594815135e-10]

In [207]:
def extract_conditions(sorted_epochs):
    
    conditions = []

    for i in sorted_epochs:

        cond = list(i.event_id.keys())
        conditions.append(cond[0])
    
    return conditions

In [208]:
conditions = extract_conditions(sorted_epochs)
conditions

['no_dis/target_l',
 'no_dis/target_r',
 'dis_right/target_l',
 'dis_left/target_r',
 'dis_top/target_l',
 'dis_top/target_r']

In [209]:
def alpha_power_df(conditions=None, right_power_list=None, left_power_list=None):
    
    df = pd.DataFrame(columns=[['condition','target_side', 'distractor_side', 'alpha_side', 'alpha_power']])
    
    df['condition'] = conditions * 2
    
    df ['alpha_power'] = right_power_list + left_power_list
    
    for row_number in range(len(df)):
                     
                        # add target side
                        if 'target_l' in df.iloc[row_number, 0]:
                            df.iloc[row_number, 1] = 'left'
                        elif 'target_r' in df.iloc[row_number, 0]:
                            df.iloc[row_number, 1] = 'right'
                        
                        # add distractor side
                        if 'dis_top' in df.iloc[row_number, 0] or 'no_dis' in df.iloc[row_number, 0]:
                            df.iloc[row_number, 2] = 'nodis'
                        elif 'dis_right' in df.iloc[row_number, 0]:
                            df.iloc[row_number, 2] = 'right'
                        elif 'dis_left' in df.iloc[row_number, 0]:
                            df.iloc[row_number, 2] = 'left'

                        # add alpha side
                        if row_number <= 5:
                            if 'target_l' in df.iloc[row_number, 0]:
                                df.iloc[row_number, 3] = 'contra'
                            elif 'target_r' in df.iloc[row_number, 0]:
                                df.iloc[row_number,3] = 'ipsi'
                        else:
                            if 'target_l' in df.iloc[row_number, 0]:
                                df.iloc[row_number,3] = 'ipsi'
                            elif 'target_r' in df.iloc[row_number, 0]:
                                df.iloc[row_number,3] = 'contra'
                        
    return df

In [210]:
df = alpha_power_df(conditions, right_power_list, left_power_list)
df

,condition,target_side,distractor_side,alpha_side,alpha_power
0,no_dis/target_l,left,nodis,contra,3.261229e-10
1,no_dis/target_r,right,nodis,ipsi,3.312846e-10
2,dis_right/target_l,left,right,contra,3.407500e-10
3,dis_left/target_r,right,left,ipsi,3.004108e-10
4,dis_top/target_l,left,nodis,contra,3.456236e-10
5,dis_top/target_r,right,nodis,ipsi,3.231706e-10
6,no_dis/target_l,left,nodis,ipsi,1.590270e-09
7,no_dis/target_r,right,nodis,contra,1.763815e-09
8,dis_right/target_l,left,right,ipsi,1.563557e-09
9,dis_left/target_r,right,left,contra,1.597437e-09
